In [1]:
! pip install transformers datasets
! pip install evaluate
! pip install sentence-transformers
! pip install git+https://github.com/huggingface/accelerate
! pip install Xformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 73.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 22.7 MB/s eta 0:00:00
Looking in index

### Προσοχή

Μη διαγράψετε τα # insert your code here σχόλια, καθώς βοηθούν στη διόρθωση. Συμπληρώστε τον κώδικά σας μετά από τα σχόλια αυτά.

# Μέρος Α: Fine-tune a pretrained model

Τα γλωσσικά μοντέλα αποτελούνται από δύο στάδια εκπαίδευσης:
1. Pre-training σε μεγάλα unlabelled datasets. Το pre-training είναι υπολογιστικά πολύ ακριβό και γι αυτό στην πράξη δε το χρησιμοποιούμε όταν θέλουμε να τρέξουμε ένα μοντέλο σε ένα καινούργιο dataset. Μπορούμε να σκεφτούμε το pre-training ως τη διαδικασία εκμάθησης γλωσσικών κανόνων κι εννοιών, οι οποίες στη συνέχεια μπορούν να χρησιμοποιηθούν για διάφορους σκοπούς.

2. Fine-tuning σε μικρότερα labelled datasets. Το fine-tuning πρακτικά εκμεταλλεύεται τις ιδιότητες του transfer learning προκειμένου να μεταφέρουμε τη 'γνώση' που έχει αποθηκευθεί στο γλωσσικό μοντέλο κατά τη διάρκεια του pre-training σε συγκεκριμένα task. Κάθε task εξυπηρετείται μέσω στοχευμένων datasets. Για παράδειγμα, κάποια datasets αναφέρονται στην ταξινόμηση κειμένων σε κατηγιορίες (text classification), άλλα datasets περιέχουν ερωτήσεις οι οποίες πρέπει να απαντηθούν (question answering) κι άλλα πολλά.

Κάποια κλασικά tasks της επεξεργασίας φυσικής γλώσσας είναι τα ακόλουθα:
- Text classification
- Question answering
- Natural language inference 
- Fill mask
- Semantic similarity

Περισσότερες πληροφορίες μπορείτε να βρείτε στον ακόλουθο σύνδεσμο στο domain Natural Language Processing: https://huggingface.co/models

Στο πρώτο κομμάτι της παρούσας εργαστηριακής άσκησης, θα χρησιμοποιήσουμε το pre-training fine-tuning σενάριο για να ταξινομήσουμε reviews.

## Pipelines

Με τη χρήση του text-classification pipeline μπορούμε να τρέξουμε γλωσσικά μοντέλα που αφορούν tasks ταξινόμησης. Το natural language inference (NLI) task αποτελεί ένα task ταξινόμησης, αφού το σχετικό μοντέλο (εν προκειμένω το roberta-large-mnli) καλείται να ταξινομήσει ένα κείμενο σε μία από τις 3 κατηγορίες [entailment/neutral/contradiction].

```
from transformers import pipeline

classifier = pipeline("text-classification", model = "roberta-large-mnli")
classifier("A soccer game with multiple males playing. Some men are playing a sport.")
## [{'label': 'ENTAILMENT', 'score': 0.98}]
```

Ένα άλλο task ταξινόμησης αφορά την αξιολόγηση του κατά πόσο ένα κείμενο είναι γραμματικά ορθό (acceptable) ή όχι (unacceptable):

```
from transformers import pipeline

classifier = pipeline("text-classification", model = "textattack/distilbert-base-uncased-CoLA")
classifier("I will walk to home when I went through the bus.")
##  [{'label': 'unacceptable', 'score': 0.95}]
```

## Σύνολο δεδομένων Yelp polarity

Κατεβάζουμε το [Yelp Polarity](https://huggingface.co/datasets/yelp_polarity) dataset το οποίο περιέχει reviews που εκφράζουν συναισθήματα πελατών για εστιατόρια. Τα reviews αυτά χωρίζονται σε κατηγορίες, και ο σκοπός μας είναι να κατηγοριοποιήσουμε νέα reviews στις σωστές κατηγορίες.

In [2]:
from datasets import load_dataset

# insert your code here
dataset = load_dataset("yelp_polarity")

Generating train split:   0%|          | 0/560000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/38000 [00:00<?, ? examples/s]

Dataset yelp_polarity downloaded and prepared to /root/.cache/huggingface/datasets/yelp_polarity/plain_text/1.0.0/14f90415c754f47cf9087eadac25823a395fef4400c7903c5897f55cfaaa6f61. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 560000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 38000
    })
})


Επειδή το σύνολο δεδομένων του Yelp Polarity περιέχει πολλά δείγματα, προκειμένου να επιταχύνουμε τη διαδικασία του fine-tuning συστήνουμε να διατηρήσετε 300 δείγματα από το train set και 300 δείγματα από το test set.

Ελέγξτε τον αριθμό κατηγοριών που υπάρχουν συνολικά στο train και το test set και διατηρήστε ισορροπημένο αριθμό δειγμάτων ανά κατηγορία για τα σύνολα αυτά κατά την επιλογή των 300 δειγμάτων.

In [3]:
# insert your code here
from datasets import Dataset


train_dataset = Dataset.from_dict(dataset['train'].shuffle()[:300])
test_dataset = Dataset.from_dict(dataset['test'].shuffle()[:300])

print(f"Train dataset has {sum(sample['label'] == 0 for sample in train_dataset)} 0s and {sum(sample['label'] == 1 for sample in train_dataset)} 1s")
print(f"Train dataset has {sum(sample['label'] == 0 for sample in test_dataset)} 0s and {sum(sample['label'] == 1 for sample in test_dataset)} 1s")

Train dataset has 135 0s and 165 1s
Train dataset has 163 0s and 137 1s


# Language Models

Η προεπεξεργασία των κειμένων προηγείται της εισόδου τους στα γλωσσικά μοντέλα. Η διαδικασία αυτή επιτελείται μέσω των Tokenizers, τα οποία μετατρέπουν τα tokens εισόδου σε κατάλληλα IDs του λεξιλογίου προεκπαίδευσης, κι έτσι μετατρέπουν το κείμενο σε μορφή που μπορεί να επεξεργαστεί κάποιο μοντέλο Transformer. Η βιβλιοθήκη Huggingface προσφέρει εύκολες και high-level υλοποιήσεις tokenization, τις οποίες συστήνουμε να ακολουθήσετε στη συνέχεια.

Συγκεκριμένα, αρχικοποιούμε τη διαδικασία του tokenization με χρήση του AutoTokenizer. Επιλέγοντας τη μέθοδο from_pretrained λαμβάνουμε έναν tokenizer που αποκρίνεται στην αρχιτεκτονική του μοντέλου που επιθυμούμε να χρησιμοποιήσουμε, παρέχοντας συμβατό tokenization.

Περισσότερες πληροφορίες για το AutoTokenization μπορείτε να βρείτε εδώ:
https://huggingface.co/docs/transformers/model_doc/auto

Αναφορικά με το μοντέλο BERT το οποίο διδαχθήκατε στο εργαστήριο, μπορείτε να δείτε τη διαδικασία [του tokenization και της αρχικοποίησης του μοντέλου](https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertTokenizer):

```
from transformers import AutoTokenizer, BertModel

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")
```

Στα πλαίσια της άσκησης καλείστε να επιτελέσετε την παραπάνω διαδικασία με *κάποιο άλλο μοντέλο της επιλογής σας από το Huggingface* που να υποστηρίζει τον AutoTokenizer. Το pre-trained μοντέλο που θα επιλέξετε θα πρέπει να διαθέτει υλοποίηση με sequence classification head (κατ αναλογία της μεθόδου BertForSequenceClassification).

Στο επόμενο κελί, φορτώστε το επιλεχθέν μοντέλο με τον αντίστοιχο tokenizer.

(Αγνοήστε πιθανά warnings της μορφής Some weights of the model checkpoint at xxx were not used when initializing...)

In [ ]:
# insert your code here

from transformers import AutoTokenizer, AlbertForSequenceClassification , RobertaForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model = RobertaForSequenceClassification.from_pretrained("roberta-base")

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classi

Σας παρέχουμε τη συνάρτηση που πραγματοποιεί το tokenization καλώντας τον tokenizer που επιλέξατε. Εφαρμόστε το τόσο στο train, όσο και στο test set.

In [ ]:
train_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 300
})

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"],padding="max_length", truncation=True)

# insert your code here
tokenized_train_dataset = Dataset.from_dict(tokenize_function(train_dataset))
tokenized_test_dataset = Dataset.from_dict(tokenize_function(test_dataset))
import pandas as pd
merged_train_dataset = Dataset.from_pandas(pd.concat([pd.DataFrame(train_dataset), pd.DataFrame(tokenized_train_dataset)],axis = 1))
merged_test_dataset = Dataset.from_pandas(pd.concat([pd.DataFrame(test_dataset), pd.DataFrame(tokenized_test_dataset)],axis = 1))
merged_test_dataset

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 300
})

Τυπώνοντας το train ή το test set, θα δείτε δύο επιπλέον πεδία 'input_ids' και 'attention_mask'. Βεβαιωθείτε ότι υπάρχουν, άρα και το tokenization έχει επιτευχθεί.

## Χρήση του PyTorch Trainer για fine-tuning

Η κλάση [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) έχει βελτιστοποιηθεί από τους δημιουργούς του Huggingface παρέχοντας πολλές διευκολύνσεις και λιγότερη 'χεράτη' δουλειά. Προτείνουμε να τη χρησιμοποιήσετε ως εναλλακτική του να γράψετε το δικό σας training loop.
Καθώς η Trainer δεν τεστάρει αυτόματα την επίδοση του εκάστοτε μοντέλου κατά την εκπαίδευση, παρέχουμε κατάλληλη συνάρτηση προκειμένου να αποτιμάται το accuracy του μοντέλου σε κάθε εποχή.

In [ ]:
import numpy as np
import evaluate
import torch
from tqdm import tqdm
from transformers import pipeline

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Η κλάση [TrainingArguments](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments) περιέχει όλες τις υπερπαραμέτρους με τις οποίες μπορείτε να πειραματιστείτε κατά τη διαδικασία fine-tuning.


Καλείστε να πειραματιστείτε με διαφορετικές υπερπαραμέτρους όπως το learning rate, batch size κλπ, καθώς επίσης και να ορίσετε optimizer και scheduler για το fine-tuning. Προτείνουμε να εκτελέσετε fine-tuning για μικρό αριθμό εποχών (άλλωστε το μοντέλο είναι ήδη προεκπαιδευμένο).

1. Θα μας δώσετε σε markdown ένα πινακάκι με διαφορετικές υπερπαραμέτρους που δοκιμάσατε και το accuracy που πετύχατε στην τελευταία εποχή. 

2. Βάσει των πειραματισμών, πώς επηρεάζουν διαφορετικές υπερπαράμετροι όπως το learning rate και το batch size το fine-tuning του μοντέλου που επιλέξατε? Σχολιάστε και αναλύστε.

In [ ]:
from transformers.optimization import AdafactorSchedule, Adafactor, AdamW, get_linear_schedule_with_warmup
from transformers import TrainingArguments, Trainer
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch import nn

args = TrainingArguments(output_dir="test_trainer",evaluation_strategy="epoch",per_device_train_batch_size=8, per_device_eval_batch_size=16 ,warmup_steps=500, learning_rate=0.0000001 ,num_train_epochs = 7.0, weight_decay=0.0005)

# insert your code here
# optimizer
optimizer = AdamW(model.parameters(), betas = (0.9, 0.999), correct_bias= True,eps= 1e-08 ,lr= 5.320000000000001e-06,weight_decay= 0.0005, no_deprecation_warning=True)
# scheduler
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=100, num_training_steps= 200)

# etc

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=merged_train_dataset,
    eval_dataset=merged_test_dataset,
    compute_metrics=compute_metrics,
    optimizers = (optimizer, scheduler)
)

Then fine-tune your model by calling [train()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.train):

In [ ]:
trained_model=trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.693213,0.513333
2,No log,0.688675,0.523333
3,No log,0.674490,0.523333
4,No log,0.445235,0.896667
5,No log,0.169947,0.956667
6,No log,0.164117,0.956667
7,No log,0.164117,0.956667


#Αποτελέσματα
per_device_train_batch_size  | per_device_eval_batch_size | learning_rate | weight_decay| Accuracy (after 7 epochs)
-----------------------------|----------------------------|---------------|-----------|-----------
8       | 16     | 1e-7   | 5e-4 | 0.9567
16      | 16     | 1e-5   | 5e-4 | 0.9533
8       | 16     | 1e-5   | 5e-4 | 0.9467
8       | 8      | 1e-6   | 5e-4 | 0.9467
8       | 8      | 1e-5   | 5e-4 | 0.94
8       | 8      | 1e-5   | 5e-3 | 0.9367
8       | 8      | 1e-5   | 5e-5 | 0.9367
8       | 8      | 1e-5   | 5e-2 | 0.9367

# Μέρος Β: Χρήση fine-tuned μοντέλων σε νέα tasks

Στο κομμάτι αυτό της εργασίας δε χρειάζεται να πραγματοποιήσετε εκπαίδευση σε γλωσσικά μοντέλα. Αντιθέτως, θα εκμεταλλευτούμε τις δυνατότητες του transfer learning για να αντιμετωπίσουμε πιο πολύπλοκα γλωσσικά task, ανάγοντάς τα σε κλασικά task όπως είναι το text classification, natural language inference, question answering και άλλα.

Για παράδειγμα, fine-tuned μοντέλα για [text classification](https://huggingface.co/tasks/text-classification) εξυπηρετούν tasks όπως:

- Είναι δύο προτάσεις η μία παράφραση της άλλης? [Paraphrase/No Paraphrase]
- Συνεπάγεται η πρόταση Χ την πρόταση Υ? [Entail/Neutral/Contradict]
- Είναι η δοθείσα πρόταση γραμματικά ορθή? [Acceptable/Unacceptable]

## B1. Piqa dataset

Το [Piqa dataset](https://huggingface.co/datasets/piqa) περιλαμβάνει προτάσεις οι οποίες ελέγχουν το βαθμό στον οποίο τα language models έχουν κοινή γνώση (commonsense). Συγκεκριμένα, αποτελείται από προτάσεις και πιθανά endings, τα οποία απαιτούν commonsense γνώση για να συμπληρωθούν.

Για παράδειγμα, έχοντας την πρόταση "When boiling butter, when it's ready, you can" υπάρχουν δύο υποψήφια endings:
- "Pour it onto a plate"
- "Pour it into a jar"

Ένας άνθρωπος μπορεί να συμπεράνει ότι η δεύτερη πρόταση αποτελεί ένα πιο κατάλληλο ending, αφού το λιωμένο βούτυρο είναι υγρό, άρα το βάζο είναι ένα καταλληλότερο δοχείο σε σχέση με το πιάτο.

Για λόγους επιτάχυνσης επιλέξτε ένα τυχαίο υποσύνολο 100 δειγμάτων από το Piqa.

In [ ]:
# # insert your code here (load dataset)
dataset = load_dataset('piqa')
piqa_dataset = Dataset.from_dict(dataset['validation'].shuffle()[:100])

  0%|          | 0/3 [00:00<?, ?it/s]

Μπορούμε να θεωρήσουμε το παραπάνω σενάριο σαν ένα πρόβλημα πολλαπλής επιλογής, όπου υπάρχουν δύο πιθανές εναλλακτικές για το ending της πρότασης. Συνεπώς, αξιοποιώντας σχετικά μοντέλα μπορούμε να επιλύσουμε την επιλογή του ending δοθείσας της πρότασης.

Καλείστε λοιπόν να καταγράψετε το accuracy πρόβλεψης endings για κάθε πρόταση με χρήση γλωσσικών μοντέλων. Για λόγους σύγκρισης χρησιμοποιήστε τουλάχιστον 5 κατάλληλα μοντέλα.

In [ ]:
# insert your code here (models)
from transformers import AutoTokenizer, BertForMultipleChoice, ElectraForMultipleChoice, AlbertForMultipleChoice, LukeForMultipleChoice, DebertaV2ForMultipleChoice, BigBirdForMultipleChoice

Bert_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
Bert_model = BertForMultipleChoice.from_pretrained("bert-base-uncased")

Electra_tokenizer = AutoTokenizer.from_pretrained("google/electra-small-discriminator")
Electra_model = ElectraForMultipleChoice.from_pretrained("google/electra-small-discriminator")

Albert_tokenizer = AutoTokenizer.from_pretrained("albert-base-v2")
Albert_model = AlbertForMultipleChoice.from_pretrained("albert-base-v2")

Luke_tokenizer = AutoTokenizer.from_pretrained("studio-ousia/luke-base")
Luke_model = LukeForMultipleChoice.from_pretrained("studio-ousia/luke-base")

DebertaV2_tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v2-xlarge")
DebertaV2_model = DebertaV2ForMultipleChoice.from_pretrained("microsoft/deberta-v2-xlarge")

BigBird_tokenizer = AutoTokenizer.from_pretrained("google/bigbird-roberta-base")
BigBird_model = BigBirdForMultipleChoice.from_pretrained("google/bigbird-roberta-base")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMultipleChoice: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly

Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraForMultipleChoice: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForMultipleChoice were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier.weight', 'sequence_summary.summ

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertForMultipleChoice: ['predictions.decoder.weight', 'predictions.dense.weight', 'predictions.LayerNorm.bias', 'predictions.dense.bias', 'predictions.bias', 'predictions.LayerNorm.weight', 'predictions.decoder.bias']
- This IS expected if you are initializing AlbertForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForMultipleChoice were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on

Some weights of the model checkpoint at studio-ousia/luke-base were not used when initializing LukeForMultipleChoice: ['lm_head.layer_norm.bias', 'entity_predictions.transform.LayerNorm.bias', 'entity_predictions.transform.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'entity_predictions.bias', 'entity_predictions.transform.dense.bias', 'lm_head.dense.weight', 'entity_predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing LukeForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LukeForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LukeForMultipleChoice were not initialized from the model c

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Some weights of the model checkpoint at microsoft/deberta-v2-xlarge were not used when initializing DebertaV2ForMultipleChoice: ['lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias']
- This IS expected if you are initializing DebertaV2ForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaV2ForMultipleChoice were not initialized from the model checkpoint at microsoft/deberta-v2-xlarge and are newly initialized: ['classifier.weight', 'pooler.dense.bias', 'p

Some weights of the model checkpoint at google/bigbird-roberta-base were not used when initializing BigBirdForMultipleChoice: ['cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BigBirdForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BigBirdForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BigBirdForMultipleChoice were not initialized from t

In [ ]:
# insert your code here (function for ending prediction)

def prediction(model, tokenizer):
  correct = 0
  for data in piqa_dataset:
    prompt = data['goal']
    choice0 = data['sol1']
    choice1 = data['sol2']
    label = torch.tensor(data['label']).unsqueeze(0)

    encoding = tokenizer([prompt, prompt], [choice0, choice1], return_tensors="pt", padding=True)
    outputs = model(**{k: v.unsqueeze(0) for k, v in encoding.items()}, labels=label)  

    loss = outputs.loss
    logits = outputs.logits
    if (torch.argmax(logits, dim=1)[0] == label):
      correct = correct + 1

  print(f"Accuracy for {str(model).split('(')[0]}: {correct/100.0}")

In [ ]:
prediction(Bert_model, Bert_tokenizer)
prediction(Electra_model, Electra_tokenizer)
prediction(Albert_model, Albert_tokenizer)
prediction(Luke_model, Luke_tokenizer)
prediction(BigBird_model, BigBird_tokenizer)

Accuracy for BertForMultipleChoice: 0.5
Accuracy for ElectraForMultipleChoice: 0.47
Accuracy for AlbertForMultipleChoice: 0.56
Accuracy for LukeForMultipleChoice: 0.43
Accuracy for BigBirdForMultipleChoice: 0.51


## B2. Truthful QA

### Sentence Transformers

Οι sentence transformers χρησιμοποιούνται για να δημιουργήσουν embeddings προτάσεων, δηλαδή διανυσματικές αναπαραστάσεις των προτάσεων αυτών σε ένα διανυσματικό χώρο. Χάρη στον τρόπο που έχουν προεκπαιδευτεί, έχουν την ικανότητα να τοποθετούν νοηματικά όμοιες προτάσεις κοντά τη μία στην άλλη, ενώ απομακρύνουν νοηματικά μακρινές προτάσεις. Έτσι, χάρη στις αναπαραστάσεις που λαμβάνουμε από τα sentence embeddings μπορούμε να αξιολογήσουμε σε τι βαθμό δύο προτάσεις είναι κοντά ή μακριά νοηματικά.

Η σύγκριση των διανυσματικών αναπαραστάσεων μπορεί να γίνει κλασικά μέσω μεθόδων όπως το consine similarity, με μεγαλύτερες τιμές μεταξύ διανυσμάτων να σηματοδοτούν πιο όμοια διανύσματα, άρα και πιο όμοιες προτάσεις. Δίνουμε για το λόγο αυτό μια συνάρτηση υπολογισμού του cosine similarity.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
def get_cosine_similarity(feature_vec_1, feature_vec_2):    
    return cosine_similarity(feature_vec_1.reshape(1, -1), feature_vec_2.reshape(1, -1))[0][0]

Για παράδειγμα, εκτελέστε το ακόλουθο κελί, το οποίο δίνει μια τιμή ομοιότητας στο διάστημα [0, 1] για δύο προτάσεις ("This is an example sentence", "Each sentence is converted"). Μπορείτε ακόμα να δοκιμάσετε να εκτελέσετε το ακόλουθο κελί για διαφορετικές προτάσεις της επιλογής σας, που μπορεί να είναι όμοιες ή πολύ διαφορετικές μεταξύ τους, και να παρατηρήσετε τις αλλαγές τιμών του cosine similarity.

In [ ]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embeddings = model.encode(sentences)

get_cosine_similarity(embeddings[0], embeddings[1])

0.40488473

Για τη συνέχεια της άσκησης, καλείστε να επιλέξετε τουλάχιστον 6 διαφορετικά [μοντέλα για semantic similarity](https://huggingface.co/models?pipeline_tag=sentence-similarity&sort=downloads) από τους sentence transformers

### Μπορούν τα question answering μοντέλα να διαχωρίσουν αληθείς και ψευδείς προτάσεις?

Αυτό το ερώτημα θα το απαντήσουμε στο παρόν κομμάτι της άσκησης. Για το λόγο αυτό, φορτώνουμε το dataset [Truthful QA generation](https://huggingface.co/datasets/truthful_qa/viewer/generation/validation), το οποίο περιέχει τις εξής επιλογές:

- best answer
- correct answer
- incorrect answer

Πολλές φορές το best answer και το correct answer είναι ίδια ή έστω πολύ κοντινά νοηματικά. Σε αυτό το σημείο είναι που θα αξιοποιήσουμε το semantic similarity για να αξιολογήσουμε την ομοιότητα αυτή. 

Φιλτράρουμε το dataset ώστε να περιέχονται 100 δείγματα συνολικά για λόγους επιτάχυνσης, εκ των οποίων καθένα θα πρέπει να περιέχει τουλάχιστον 2 correct answer. Θεωρούμε έτσι 4 υποψήφιες επιλογές:

1η επιλογή: best answer  
2η επιλογή: 1ο correct answer  
3η επιλογή: 2ο correct answer  
4η επιλογή: incorrect answer  

Οι επιλογές αυτές μαζί με την ερώτηση δίνονται σε ένα μοντέλο πολλαπλής επιλογής σαν αυτά που χρησιμοποιήθηκαν στο ερώτημα Β1. Μπορείτε να θεωρήσετε τα ίδια μοντέλα και να τα επεκτείνετε σε 4 υποψήφιες απαντήσεις.  

Το semantic similarity θα επηρεάσει το τι θεωρούμε βέλτιστα σωστή απάντηση, άρα και το accuracy. Συγκεκριμένα, θα λάβουμε διανυσματικές αναπαραστάσεις για το best answer και τα 2 correct answer που έχουν δοθεί ως υποψήφιες επιλογές μέσω κάποιου semantic similarity μοντέλου. Σε περίπτωση λοιπόν που το μοντέλο πολλαπλής επιλογής προβλέψει ένα εκ των correct answer, και η ομοιότητά τους σε σχέση με το best model ξεπερνάει ένα προεπιλεγμένο κατώφλι ομοιότητας, η απάντηση θεωρείται βέλτιστα σωστή. Θέτουμε λοιπόν κατώφλι ομοιότητας το 0.95.

Για παράδειγμα, έστω ότι το μοντέλο πολλαπλής επιλογής μεταξύ των υποψηφίων [best, 1st correct, 2nd correct, incorrect] επιλέγει το δεύτερο στοιχείο, δηλαδή το 1st correct, και δεδομένου ότι το cosine similarity μεταξύ των embeddings του best και του 1st correct είναι > 0.95, τότε θεωρούμε ότι η απάντηση είναι βέλτιστα σωστή, και συνυπολογίζεται θετικά στο accuracy.

Καλείστε λοιπόν να γράψετε μια συνάρτηση που να υπολογίζει το accuracy εύρεσης βέλτιστα σωστών απαντήσεων ανάμεσα στις υποψήφιες απαντήσεις, εξετάζοντας τουλάχιστον 6 semantic similarity μοντέλα καθώς επίσης και τα μοντέλα που επιλέξατε στο ερώτημα Β1.


In [ ]:
# insert your code here (load dataset)
dataset = load_dataset('truthful_qa','generation')['validation']
truthful_qa_dataset = Dataset.from_dict(Dataset.from_list([dataset[i] for i in range(dataset.num_rows) if len(dataset[i]["correct_answers"]) >= 2]).shuffle()[:100])

truthful_qa_dataset = truthful_qa_dataset.map(lambda dt: {**dt, 'correct_1': dt['correct_answers'][0], 'correct_2': dt['correct_answers'][1], 'incorrect': dt['incorrect_answers'][0]})
truthful_qa_dataset = truthful_qa_dataset.remove_columns(['type', 'category','source', 'correct_answers', 'incorrect_answers'])

print(truthful_qa_dataset)

Generating validation split:   0%|          | 0/817 [00:00<?, ? examples/s]

Dataset truthful_qa downloaded and prepared to /root/.cache/huggingface/datasets/truthful_qa/generation/1.1.0/70210b72382652635215516e59663843b88eda16bd2acef909fb46700beb039a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Dataset({
    features: ['question', 'best_answer', 'correct_1', 'correct_2', 'incorrect'],
    num_rows: 100
})


In [ ]:
# insert your code here (load models for semantic similarity and QA)

semantic_model_1 = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
semantic_model_2 = SentenceTransformer('sentence-transformers/multi-qa-mpnet-base-dot-v1')
semantic_model_3 = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')
semantic_model_4 = SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L6-v2')
semantic_model_5 = SentenceTransformer('sentence-transformers/bert-base-nli-mean-tokens')
semantic_model_6 = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2')

In [ ]:
# insert your code here (function for optimal correct answers & semantic similarity)

def semantic_similarity(multiple_choice_model, multiple_choice_tokenizer, semantic_model, model_name):
  correct = 0
  for data in truthful_qa_dataset:
    question = data['question']
    best = data['best_answer']
    correct_1 = data['correct_1']
    correct_2 = data['correct_1']
    incorrect = data['incorrect']
    options = [best, correct_1, correct_2, incorrect]

    encoding = multiple_choice_tokenizer(4*[question], options, return_tensors="pt", padding=True)
    output = multiple_choice_model(**{k: v.unsqueeze(0) for k, v in encoding.items()})  

    embeddings = semantic_model.encode([best, options[torch.argmax(output.logits, dim=1)[0]]])

    if (get_cosine_similarity(embeddings[0], embeddings[1]) >= 0.95):
      correct = correct + 1

  print(f"Accuracy for {str(multiple_choice_model).split('(')[0]} with {model_name}: {correct/100.0}")

In [ ]:
semantic_models = [semantic_model_1, semantic_model_2, semantic_model_3, semantic_model_4, semantic_model_5, semantic_model_6]
semantic_model_names = ['all-MiniLM-L6-v2', 'multi-qa-mpnet-base-dot-v1', 'all-MiniLM-L12-v2', 'paraphrase-MiniLM-L6-v2', 'bert-base-nli-mean-tokens', 'distiluse-base-multilingual-cased-v2']
multiple_choice_models = [Bert_model, Electra_model, Albert_model, Luke_model, BigBird_model]
tokenizers = [Bert_tokenizer, Electra_tokenizer, Albert_tokenizer, Luke_tokenizer, BigBird_tokenizer]

for i in range(5):
  for j in range(6):
    semantic_similarity(multiple_choice_models[i], tokenizers[i], semantic_models[j], semantic_model_names[j])
  print('----------------------------------------')

Accuracy for BertForMultipleChoice with all-MiniLM-L6-v2: 0.54
Accuracy for BertForMultipleChoice with multi-qa-mpnet-base-dot-v1: 0.55
Accuracy for BertForMultipleChoice with all-MiniLM-L12-v2: 0.54
Accuracy for BertForMultipleChoice with paraphrase-MiniLM-L6-v2: 0.51
Accuracy for BertForMultipleChoice with bert-base-nli-mean-tokens: 0.52
Accuracy for BertForMultipleChoice with distiluse-base-multilingual-cased-v2: 0.52
----------------------------------------
Accuracy for ElectraForMultipleChoice with all-MiniLM-L6-v2: 0.55
Accuracy for ElectraForMultipleChoice with multi-qa-mpnet-base-dot-v1: 0.55
Accuracy for ElectraForMultipleChoice with all-MiniLM-L12-v2: 0.55
Accuracy for ElectraForMultipleChoice with paraphrase-MiniLM-L6-v2: 0.54
Accuracy for ElectraForMultipleChoice with bert-base-nli-mean-tokens: 0.53
Accuracy for ElectraForMultipleChoice with distiluse-base-multilingual-cased-v2: 0.54
----------------------------------------
Accuracy for AlbertForMultipleChoice with all-Mini

Attention type 'block_sparse' is not possible if sequence_length: 23 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...


Accuracy for LukeForMultipleChoice with distiluse-base-multilingual-cased-v2: 0.5
----------------------------------------
Accuracy for BigBirdForMultipleChoice with all-MiniLM-L6-v2: 0.48
Accuracy for BigBirdForMultipleChoice with multi-qa-mpnet-base-dot-v1: 0.49
Accuracy for BigBirdForMultipleChoice with all-MiniLM-L12-v2: 0.49
Accuracy for BigBirdForMultipleChoice with paraphrase-MiniLM-L6-v2: 0.45
Accuracy for BigBirdForMultipleChoice with bert-base-nli-mean-tokens: 0.46
Accuracy for BigBirdForMultipleChoice with distiluse-base-multilingual-cased-v2: 0.46
----------------------------------------


## Β3. Winogrande dataset

Το [Winogrande dataset](https://huggingface.co/datasets/winogrande) αποτελείται από προτάσεις που μία λέξη τους έχει αφαιρεθεί και δίνονται δύο πιθανές επιλογές συμπλήρωσης του κενού. Για παράδειγμα, δοθείσας της πρότασης "John moved the couch from the garage to the backyard to create space. The _ is small.", υπάρχουν δύο πιθανές εναλλακτικές:

- "garage"
- "backyard"

Η δυσκολία της συμπλήρωσης έγκειται στο ότι και οι δύο λέξεις αναφέρονται στην πρόταση, οπότε το μοντέλο θα πρέπει να διαθέτει υψηλές δυνατότητες κατανόησης γλώσσας προκειμένου να επιλέξει μια νοηματικά σωστή συμπλήρωση. 

Για λόγους επιτάχυνσης, επιλέξτε ένα τυχαίο υποσύνολο 100 δειγμάτων από το training set του Winogrande.


In [4]:
# insert your code here (load dataset)

dataset = load_dataset('winogrande', 'winogrande_xl')['validation']

winogrande_dataset = Dataset.from_dict(dataset.shuffle()[:100])

sentence1_list = []
sentence2_list = []
label_list = []

for row in winogrande_dataset:    
    sentence1_list.append(row['sentence'].split('_')[0].strip())
    sentence1_list.append(row['sentence'].split('_')[0].strip())

    sentence2_list.append(row['option1'].strip() + ' ' + row['sentence'].split('_')[1].strip())
    sentence2_list.append(row['option2'].strip() + ' ' + row['sentence'].split('_')[1].strip())

    label_list.append(row['answer'] == '1')
    label_list.append(row['answer'] == '2')

winogrande = Dataset.from_dict({
    'sentence1': sentence1_list,
    'sentence2': sentence2_list,
    'label': label_list
})

print(winogrande)


Generating train split:   0%|          | 0/40398 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1767 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1267 [00:00<?, ? examples/s]

Dataset winogrande downloaded and prepared to /root/.cache/huggingface/datasets/winogrande/winogrande_xl/1.1.0/a826c3d3506aefe0e9e9390dcb53271070536586bab95849876b2c1743df56e2. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Dataset({
    features: ['sentence1', 'sentence2', 'label'],
    num_rows: 200
})


Με κατάλληλο [μετασχηματισμό](https://huggingface.co/DeepPavlov/roberta-large-winogrande) της παραπάνω εισόδου (πρόταση με κενό και δύο επιλογές συμπλήρωσης), καλείστε να καταγράψετε το accuracy σχετικών μοντέλων που επιλύουν το πρόβλημα, συγκρίνοντας το predicted label με το πραγματικό label (1: πρώτη επιλογή, 2: δεύτερη επιλογή). Ουσιαστικά θα πρέπει να αναγάγετε το παραπάνω πρόβλημα σε κάποιο πιο κλασικό πρόβλημα της επεξεργασίας φυσικής γλώσσας.

Δοκιμάστε τουλάχιστον 3 κατάλληλα μοντέλα από το Huggingface για να προσεγγίσετε το πρόβλημα του Winogrande. Προτείνουμε τη χρήση pipelines για τη διευκόλυνσή σας.

In [13]:
# insert your code here (load models)
from transformers import AutoTokenizer, AutoModelForSequenceClassification

roberta_tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/roberta-large-winogrande")
roberta_model = AutoModelForSequenceClassification.from_pretrained("DeepPavlov/roberta-large-winogrande")

deberta_tokenizer = AutoTokenizer.from_pretrained("sileod/deberta-v3-base-tasksource-nli")
deberta_model = AutoModelForSequenceClassification.from_pretrained("sileod/deberta-v3-base-tasksource-nli")

bert_tokenizer = AutoTokenizer.from_pretrained("bpark5233/bert-base-uncased-finetuned-winogrande_debiased")
bert_model = AutoModelForSequenceClassification.from_pretrained("bpark5233/bert-base-uncased-finetuned-winogrande_debiased")


In [14]:
# insert your code here (create pipelines)
from transformers import pipeline
classifier_roberta = pipeline(model = "DeepPavlov/roberta-large-winogrande", tokenizer = roberta_tokenizer)
classifier_deberta = pipeline(model = "sileod/deberta-v3-base-tasksource-nli" , tokenizer = deberta_tokenizer)
classifier_bert = pipeline(model = "bpark5233/bert-base-uncased-finetuned-winogrande_debiased", tokenizer = bert_tokenizer )


In [15]:
# insert your code here (function for predicting best fill)
def predictions(classifier, name):
  labels = []
  for i in range(0,200,2):
    sentence_1 = classifier(winogrande['sentence1'][i] + winogrande['sentence2'][i])
    sentence_2 = classifier(winogrande['sentence1'][i+1] + winogrande['sentence2'][i+1])
    if(sentence_1[0]['label'] == 'True' and sentence_2[0]['label'] == 'False'):
      labels.append('1')
    elif(sentence_1[0]['label'] == 'False' and sentence_2[0]['label'] == 'True'):
      labels.append('2')
    elif(sentence_1[0]['label'] == 'False' and sentence_2[0]['label'] == 'False'):
      labels.append('1' if sentence_1[0]['score']<sentence_2[0]['score'] else '2')
    else:
      labels.append('1' if sentence_1[0]['score']>sentence_2[0]['score'] else '2')

  k = sum(labels[i] == winogrande_dataset['answer'][i] for i in range(100)) 
  print(f'Accuracy for {name} : {k/100}' ) 


In [16]:
predictions(classifier_roberta,"DeepPavlov/roberta-large-winogrande")
predictions(classifier_deberta,"sileod/deberta-v3-base-tasksource-nli")
predictions(classifier_bert,"bpark5233/bert-base-uncased-finetuned-winogrande_debiased")

Accuracy for DeepPavlov/roberta-large-winogrande : 0.78
Accuracy for sileod/deberta-v3-base-tasksource-nli : 0.57
Accuracy for bpark5233/bert-base-uncased-finetuned-winogrande_debiased : 0.54
